In [1]:
import xlearn as xl
import pandas as pd
import numpy as np

from models.base import create_user, site_app_split

## `totoruo/FfmEncoder`
Gets stuck at 'converting data...' at line 353.
Try `Bobe24/Dataframe2libffm`.

In [2]:
df = pd.read_csv('./data/train_tiny.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)
df_site, df_app = site_app_split(df)

In [3]:
categorical_features = ['banner_pos', 'platform_id', 'platform_category',
                        'user', 'device_conn_type', 'C14','C17','C20','C21']

In [4]:
from tools.FfmEncoder import FfmEncoder

In [5]:
encoder = FfmEncoder(categorical_features, label_name='click', nthread=2)

In [6]:
encoder.transform(df_app, 'train_app.ffm')

converting data......


In [7]:
raise

RuntimeError: No active exception to reraise

## LibFFM conversion stats: memory issue
With categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
                        'user', 'device_conn_type', 'C14','C17','C20','C21']
- just a few seconds to build feature_dict for app-small.
- 12s for app-mid. len = 0.8M
- 21s for site-mid. len = 1.6M
- encode_features exhausts memory...

In [ ]:
def make_field_dict(df, fields):
    """
    fields: Array[String] - a list of column names
    A field dictionary is just an inverted column index.
    """
    return {col: i for i, col in enumerate(fields)}

def make_feature_dict(df, fields):
    # prepend a field name to each feature in order to distinguish 
    # a feature name present in two or more fields.
    for c in fields:
        df[c] = f'{c}_' + df[c].astype('str')
    # TODO: we could hash all features at this stage.
    # TODO: hash into a smaller space to make the dict smaller
    df[fields] = df[fields].applymap(hash)
    # Index features from all fields.
    features_concat = pd.concat([df[c] for c in fields], ignore_index=True)
    uniques = features_concat.unique()
    return pd.Series(np.arange(len(uniques)),index=uniques)

def encode_features(df, feature_dict):
    return df.replace(feature_dict)

def encode_features_old(df, feature_dict, fields):
    # df.replace(feature_dict) doesn't fit in memory.
    # optimize by splitting the feature_dict into dictionaries corresponding to fields.
    replace_dict = dict()
    for c in fields:
        replace_dict[c] = {k: v for k, v in feature_dict.items() 
                           if k.startswith(c)}
    
    return df.replace(replace_dict)

In [ ]:
df = pd.read_csv('./data/train_small.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)
df_site, df_app = site_app_split(df)

In [ ]:
categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
                        'user', 'device_conn_type', 'C14','C17','C20','C21']

In [ ]:
field_dict = make_field_dict(df_site, categorical_features)
feature_dict = make_feature_dict(df_site, categorical_features)

In [ ]:
%%time
df_site_encoded = encode_features(df_site, feature_dict)

In [ ]:
len(feature_dict)

In [ ]:
raise

## Develop LibFFM format converter
Refer to the Data Format section of https://github.com/guestwalk/libffm .

In [ ]:
df = pd.read_csv('./data/train_tiny.csv')
categorical_features = ['C1','device_type']
df.head()

In [ ]:
field_dict = make_field_dict(df, categorical_features)
feature_dict = make_feature_dict(df, categorical_features)

In [ ]:
feature_dict

In [ ]:
df_encoded = encode_features(df, feature_dict)

## Quickstart
xl has ffm, fm, and linear models. ffm models only take the libffm format, while others take csv or the libsvm format.

In [ ]:
ffm_model = xl.create_ffm()

In [ ]:
ffm_model.setTrain('./data/small_train.txt')

In [ ]:
# param:
#  0. task: binary classification
#  1. learning rate : 0.2
#  2. regular lambda : 0.002
param = {'task':'binary', 'lr':0.2, 'lambda':0.002}

In [ ]:
ffm_model.fit(param, "./model.out")

In [ ]:
ffm_model.setTest('./data/small_test.txt')

In [ ]:
ffm_model.predict('./model.out', './xlearn-output.txt')

## xlearn with its sklearn API (deprecated)
FFMModel.fit takes array-like. The format of its fields argument is unclear.

In [ ]:
ffm = xl.FFMModel(task='binary', lr=0.2, epoch=10, reg_lambda=0.002, metric='acc')
#ffm.fit('./data/small_train.txt')